In [56]:
using JuMP
using Cbc
import Statistics
using DataFrames
using CSV
using Gurobi
using Ipopt

Data1000 = DataFrame(CSV.File("C:\\Users\\mgharibdousti\\Desktop\\Dissertation\\Data1000.dat"))
LT1000 = DataFrame(CSV.File("C:\\Users\\mgharibdousti\\Desktop\\Dissertation\\LT1000.dat"))
Distance= DataFrame(CSV.File("C:\\Users\\mgharibdousti\\Desktop\\Dissertation\\Distance1000.dat"))
FINAL_MELD=Data1000[3]
INIT_MELD=Data1000[4]
FINAL_MELD=Data1000[3]
INIT_MELD=Data1000[4]
WL=Data1000[5]
WT=Data1000[7]
LDRI=Data1000[8]

#N = 1000
#NN = 1000
#alpha= 1.1
#w = rand(NN,N)
#sigma= rand(NN,N)
K=1
#p=rand(K)
#q=rand(NN,N,K)

    function f(N,alpha,f1,f2,p1)
    w = rand(N,N)
    sigma= rand(N,N)
    p=rand(K)
    q=rand(N,N,K)
    t=rand(N)
    h=rand(N)
    m=Model(with_optimizer(Gurobi.Optimizer))
    @variable(m, x[1:N,1:N], Bin)
    @variable(m, y[1:N,1:N,1:K], Bin)
    #@variable(m, z[1:N,1:l], Bin)



    for i in 1:N
        for j in 1:N
            w[i,j]= f1*(2.5*LT1000[i,j]-WL[j]) - f2*(WT[j])
        end
    end 


    for j in 1:N
        @constraint(m,  sum(x[i,j] for i=1:N)<=1.0)
    end

    for i in 1:N
        @constraint(m,  sum(x[i,j] for j=1:N)<=1.0)
    end
    
    for i in 1:N
        for j in 1:N
            for k in 1:K
                @constraint(m, x[i,j]<= y[i,j,k])
            end
        end
    end 

    for i in 1:N
        for j in 1:N
            @constraint(m,sum(y[i,j,k] for k=1:K) <= N)
        end
    end
    
    for k in 1:K
        for j in 1:N
            @constraint(m,  sum(y[i,j,k] for i=1:N)<=1.0)
        end
    end

    for k in 1:K
        for i in 1:N
            @constraint(m,  sum(y[i,j,k] for j=1:N)<=1.0)
        end
    end
    
    
    
    for i in 1:N
        for j in 1:N
            @constraint(m,(y[i,j,1]) <= N)
        end
    end
            


    for i in 1:N
        for j in 1:N
            sigma[i,j]= (LDRI[i]/t[i])/(FINAL_MELD[j]/h[j])
        end
    end   
    
   
    for i in 1:N
        t[i] =Statistics.max(LDRI[i])-Statistics.min(LDRI[i])
    end
    
  
      for j in 1:N
        h[j]= Statistics.max(FINAL_MELD[j])-Statistics.min(FINAL_MELD[j])
    end

    c=100
    for i in 1:N
        for j in 1:N
            q[i,j,1] = 1 - exp((sigma[i,j])/c)
        end
    end
     
    
    @objective(m, Max, sum((w[i,j]-(alpha)*Statistics.mean(w))*x[i,j] for i=1:N, j=1:N)+ 
    sum(p1*q[i,j,1]*y[i,j,1] for i=1:N, j=1:N))

    #@objective(m, Max, sum(p[k]*q[i,j,k]*y[i,j,k] for  k=1:K , i=1:NN, j=1:N))

    optimize!(m)

    NumberOfMatched=0
    for i in 1:N
        for j in 1:N
            if value.(x[i,j]) ==1
                NumberOfMatched += 1
            end
        end
    end

    NumberOfMatchedinFirstScenario=0
    for i in 1:N
        for j in 1:N
            if value.(y[i,j,1]) ==1
                NumberOfMatchedinFirstScenario += 1
            end
        end
    end

            
TransplantBenefit=0
    for i in 1:N
        for j in 1:N
            TransplantBenefit += floor((2.5*LT1000[i,j]-WL[j])*(value.(x[i,j])))
        end
    end

    WaitingTime=0
    for i in 1:N
        for j in 1:N
            WaitingTime += floor((WT[j])*(value.(x[i,j])))
        end
    end
    
    TotalNumberOfMatched=0
    for i in 1:N
        for j in 1:N
            if value.(y[i,j,1]) ==1 || value.(x[i,j]) ==1 
                TotalNumberOfMatched += 1
            end
        end
    end
    


    println("Number of Matched pairs is:", NumberOfMatched)
    println("Number of Matched pairs in First Scenario is:",  NumberOfMatchedinFirstScenario)
    println("Transplant Benefit is:", TransplantBenefit)
    println("Waiting Time is:", WaitingTime)
    
    return TransplantBenefit, WaitingTime, TotalNumberOfMatched 
end



ArgumentError: ArgumentError: syntax df[column] is not supported use df[!, column] instead

In [53]:
#N=[5,10,15,25,50,75,100,125,150,200]
#N=[1,25,5,10,50]
#alpha1=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.5]
#p1=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
p1= 0.2
N=50
f1= 0.2
f2= 0.25
E=5
alpha=0
TotalNumberOfMatched= fill(0, (E))
TransplantBenefit = fill(0, (E))
WaitingTime = fill(0, (E))

for i in 1:E
    TotalNumberOfMatched[i]=f(N[i],alpha,f1,f2,p1)[3]
end


for i in 1:E
    TransplantBenefit[i]= f(N[i],alpha,f1,f2,p1)[1]
end

for i in 1:E
    WaitingTime[i]= f(N[i],alpha,f1,f2,p1)[2]
end

Academic license - for non-commercial use only - expires 2021-04-16
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 2 columns and 8 nonzeros
Model fingerprint: 0xc2476938
Variable types: 0 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 1287.4362141
Presolve removed 7 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 1287.44 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.287436214096e+03, best bound 1.287436214096e+03, gap 0.0000%

User-callback calls 36, time in user-callback 0.00 sec


BoundsError: BoundsError: attempt to access 8-element Array{AbstractArray{T,1} where T,1} at index [9]

In [55]:
f(50,0.2,0.7,0.3,0.3)

Academic license - for non-commercial use only - expires 2021-04-16


BoundsError: BoundsError: attempt to access 8-element Array{AbstractArray{T,1} where T,1} at index [9]

In [12]:
import Pkg